In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime
import json
import requests
import csv
import os
from collections import OrderedDict
from IPython.display import display

In [200]:
def unique_list(l):
    list_set = set(l) 
    return list(list_set)
        
def get_sp_companies():
    data = pd.read_csv('sp500.csv')
    return data
    
# get stock data from yahoo finance
def get_stock_data(tickers, start=None, end=None, interval='1d', period=None):
    return yf.download(
        tickers=' '.join(tickers),
        start=start,
        end=end,
        period=period,
        interval=interval,
        group_by="ticker",
        auto_adjust=True,
        threads=True
    )

def get_period_performance(data, periods=[360], stat='Close', offset=0, weight=1):
    tickers = unique_list([x[0] for x in data.columns.values])
    columns = periods
    if offset > 0:
        columns += [f"Next_{offset}"]
    df = pd.DataFrame(columns=columns, index=tickers)

    for ticker in tickers:
        period_pctchanges = []
        for i, period in enumerate(periods):
            if offset > 0:
                chunk = data.iloc[-(period+offset):-offset]
            else:
                chunk = data.iloc[-(period+offset):]
            
            first_close = chunk[ticker].iloc[0]['Close']
            last_close = chunk[ticker].iloc[-1]['Close']
            period_pctchanges.append(((last_close - first_close) / first_close) * 100)
         
        df.loc[ticker] = period_pctchanges
        display(df)
            
    
# calculate performance data from dataframes
# using the periods and offset
def get_performance(data, periods=[360], stat='Close', offset=0):
    perf_data = {}
    for i, period in enumerate(periods):
        for t, d in data.iteritems():
            if t[1] == stat:
                chunk = None
                if offset > 0:
                    chunk = d[-(period+offset):-offset]
                else:
                    chunk = d[-(period+offset):]

                if not t[0] in perf_data:
                    perf_data[t[0]] = OrderedDict()

                perf_data[t[0]][period] = \
                    {'raw': (chunk[-1] - chunk[0]) / chunk[0] * 100}            
        
                #check if offset is used and we're at the last data period
                if offset > 0 and i == len(periods)-1:
                    new_chunk = d[-(offset):]
                    perf_data[t[0]][offset]['next_period'] = \
                        (new_chunk[-1] - new_chunk[0]) / new_chunk[0] * 100

    return perf_data

# weight the performance data based on the periods used
def get_weighted_performance(d, weight=1):
    for ticker, period in d.items():
        for period_key, perf_data in d[ticker].items():
            d[ticker][period_key]['weighted'] = \
                (weight / int(period_key)) * perf_data['raw']

    return d

# rank investements and calculate performance of next period
def get_new_investments(d):
    weighted_totals = []
    for ticker, perf_data in d.items():
        ticker_total = 0
        next_period = 0
        for period, data in perf_data.items():
            ticker_total += data['weighted']
            if 'next_period' in data:
                next_period = data['next_period']
        
        weighted_totals.append((ticker, \
            np.around(ticker_total, decimals=2), np.around(next_period, decimals=2,)))

    return weighted_totals

# sort tuples
def sort_list_of_tuples(l, reverse=True):  
    return(sorted(l, key = lambda x: x[1], reverse=reverse))


In [201]:
data_set_intervals = [
        #[144, 89, 34],
        #[144, 34, 8],
        #[34, 21, 13, 8, 5],
        #[89, 34, 13, 5],
        #[144, 55, 21, 8],
        [89, 21, 5],
    ]
sp500 = get_sp_companies()
#data = get_stock_data(tickers, period='2y', interval='1d')

In [202]:
data = None
data_file = 'sp500_data.pkl'
if os.path.exists(data_file):
    data = pd.read_pickle(data_file)
else:
    data = get_stock_data(list(sp500['Symbol'].values) + ['SPY'], period='2y', interval='1d')
    data.to_pickle(data_file)

In [203]:
pf_data = get_period_performance(data, [89, 21, 5], )
display(pf_data)

,89,21,5
NBL,-63.7549,31.723,19.242
PWR,NaN,NaN,NaN
EBAY,NaN,NaN,NaN
AEP,NaN,NaN,NaN
DLTR,NaN,NaN,NaN
...,...,...,...
TXN,NaN,NaN,NaN
OMC,NaN,NaN,NaN
SO,NaN,NaN,NaN
WELL,NaN,NaN,NaN


,89,21,5
NBL,-63.7549,31.723,19.242
PWR,-17.4094,8.08261,1.0107
EBAY,NaN,NaN,NaN
AEP,NaN,NaN,NaN
DLTR,NaN,NaN,NaN
...,...,...,...
TXN,NaN,NaN,NaN
OMC,NaN,NaN,NaN
SO,NaN,NaN,NaN
WELL,NaN,NaN,NaN


,89,21,5
NBL,-63.7549,31.723,19.242
PWR,-17.4094,8.08261,1.0107
EBAY,10.1128,30.4708,4.54304
AEP,NaN,NaN,NaN
DLTR,NaN,NaN,NaN
...,...,...,...
TXN,NaN,NaN,NaN
OMC,NaN,NaN,NaN
SO,NaN,NaN,NaN
WELL,NaN,NaN,NaN


,89,21,5
NBL,-63.7549,31.723,19.242
PWR,-17.4094,8.08261,1.0107
EBAY,10.1128,30.4708,4.54304
AEP,-10.1011,1.74817,0.38596
DLTR,NaN,NaN,NaN
...,...,...,...
TXN,NaN,NaN,NaN
OMC,NaN,NaN,NaN
SO,NaN,NaN,NaN
WELL,NaN,NaN,NaN


,89,21,5
NBL,-63.7549,31.723,19.242
PWR,-17.4094,8.08261,1.0107
EBAY,10.1128,30.4708,4.54304
AEP,-10.1011,1.74817,0.38596
DLTR,-19.0135,-4.60711,-5.87196
...,...,...,...
TXN,NaN,NaN,NaN
OMC,NaN,NaN,NaN
SO,NaN,NaN,NaN
WELL,NaN,NaN,NaN


...

,89,21,5
NBL,-63.7549,31.723,19.242
PWR,-17.4094,8.08261,1.0107
EBAY,10.1128,30.4708,4.54304
AEP,-10.1011,1.74817,0.38596
DLTR,-19.0135,-4.60711,-5.87196
...,...,...,...
TXN,NaN,NaN,NaN
OMC,NaN,NaN,NaN
SO,NaN,NaN,NaN
WELL,NaN,NaN,NaN


,89,21,5
NBL,-63.7549,31.723,19.242
PWR,-17.4094,8.08261,1.0107
EBAY,10.1128,30.4708,4.54304
AEP,-10.1011,1.74817,0.38596
DLTR,-19.0135,-4.60711,-5.87196
...,...,...,...
TXN,NaN,NaN,NaN
OMC,NaN,NaN,NaN
SO,NaN,NaN,NaN
WELL,NaN,NaN,NaN


,89,21,5
NBL,-63.7549,31.723,19.242
PWR,-17.4094,8.08261,1.0107
EBAY,10.1128,30.4708,4.54304
AEP,-10.1011,1.74817,0.38596
DLTR,-19.0135,-4.60711,-5.87196
...,...,...,...
TXN,NaN,NaN,NaN
OMC,NaN,NaN,NaN
SO,NaN,NaN,NaN
WELL,NaN,NaN,NaN


,89,21,5
NBL,-63.7549,31.723,19.242
PWR,-17.4094,8.08261,1.0107
EBAY,10.1128,30.4708,4.54304
AEP,-10.1011,1.74817,0.38596
DLTR,-19.0135,-4.60711,-5.87196
...,...,...,...
TXN,NaN,NaN,NaN
OMC,NaN,NaN,NaN
SO,NaN,NaN,NaN
WELL,NaN,NaN,NaN


KeyboardInterrupt: 

In [49]:
print("Sector strength according to STOCKINATOR")
print(f"Date of report: {datetime.datetime.now()}")

for days_back in data_set_intervals:
    print()
    print(f"Intervals and weighting used: {days_back}")
    
    #offset = 0
    #offset = days_back[-1]
    offset = days_back[len(days_back)-2]
    raw_perf_data = get_performance(data, periods=days_back, offset=offset)
    weighted_perf_data = get_weighted_performance(raw_perf_data, weight=days_back[-1])
    new_stocks = get_new_investments(weighted_perf_data)
    
    print(f"Ticker, Rank, Performance Next {offset}")
    sorted_stocks = sort_list_of_tuples(new_stocks)
    for stock in sorted_stocks:
        print(f"{stock[0]}, {stock[1]}, {stock[2]}")

Sector strength according to STOCKINATOR
Date of report: 2020-04-26 11:08:21.580706

Intervals and weighting used: [89, 21, 5]
Ticker, Rank, Performance Next 21
EVHC, nan, nan
WYNN, 51.46, 7.13
AAL, 39.57, -34.16
CCL, 39.24, -33.16
NBL, 37.08, 31.72
LLL, nan, nan
HAS, 35.04, 9.64
ALGN, 22.85, 0.17
UNM, 22.37, -1.44
HAL, 22.36, 25.67
ULTA, 22.24, 11.49
MOS, 20.8, 5.69
CNP, 12.66, 4.97
LOW, 12.08, 12.73
MU, 12.01, -1.41
AMD, 11.53, 18.27
TEL, 8.27, 2.86
ROK, 8.21, 9.61
PKI, 7.88, 10.91
PWR, 6.84, 8.08
KSU, 6.46, -1.55
DLTR, 3.0, -4.61
TTWO, 2.57, 10.96
APH, 2.37, 8.84
CHTR, 1.93, 11.97
CI, 1.87, 11.01
CMI, 1.54, 9.49
MET, 0.06, 2.37
KHC, -0.5, 18.01
AMP, -1.4, -3.06
HII, -1.49, -5.53
LH, -2.22, 19.46
VRSN, -2.67, 20.6
PEG, -3.44, 16.72
SHW, -4.09, 8.56
DLR, -4.31, 11.85
LEG, -4.5, -1.42
NI, -7.23, 6.03
XYL, -7.6, 0.75
GPC, -7.91, 9.31
DGX, -8.4, 20.76
AWK, -8.45, 4.77
TXT, -9.22, -7.53
BAX, -10.03, 15.9
EBAY, -10.53, 30.47
JNJ, -10.63, 22.35
MAA, -10.69, 4.48
FAST, -11.11, 11.51
HRL, -12